<a href="https://colab.research.google.com/github/mit1280/LLM-Agent/blob/main/CrewAI_Ollama_Sequential_Agent_Free_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,584 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,227 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/u

In [ ]:
!pip install --q U crewai crewai-tools
# !pip install --q duckduckgo-search==5.0 pyngrok
# !pip install --q langchain-huggingface langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.9/514.9 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 47

## Import Libraries

In [ ]:
import os
from crewai import Agent, LLM, Crew, Process, Task
import threading
import subprocess
import time
# from langchain.agents import Tool
# from langchain.agents import load_tools
# from langchain.llms import Ollama
# from langchain.tools import DuckDuckGoSearchRun

## Start Ollama process

In [ ]:
def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
!ollama pull phi3.5

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling b5374915da53...   0% ▕▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕▏

## Run Sequential Agent using CrewAI.

In [ ]:
# load LLM
llm = LLM(
    model="ollama/phi3.5",
    temperature=0.7,
    api_base="http://localhost:11434",
)

In [ ]:
# Define 3 agents
researcher = Agent(
  role='Researcher',
  goal='Conduct foundational research',
  backstory='An experienced researcher with a passion for uncovering insights',
  llm=llm,
  model_provider="ollama",
  verbose=True
)
analyst = Agent(
  role='Data Analyst',
  goal='Analyze research findings',
  backstory='A meticulous analyst with a knack for uncovering patterns',
  llm=llm,
  model_provider="ollama",
  verbose=True
)
writer = Agent(
  role='Writer',
  goal='Draft the final report',
  backstory='A skilled writer with a talent for crafting compelling narratives',
  llm=llm,
  model_provider="ollama",
  verbose=True
)

In [ ]:
# Define 3 tasks of each agent
research_task = Task(
  description='Generate relevant 50 rows about 2-3 random stocks in csv formate...',
  agent=researcher,
  expected_output='Raw Data'
)
analysis_task = Task(
  description='Analyze the data...',
  agent=analyst,
  expected_output='Data Insights'
)
writing_task = Task(
  description='Compose the report...',
  agent=writer,
  expected_output='Final Report'
)

In [ ]:
# Form the crew with a sequential process
report_crew = Crew(
  agents=[researcher, analyst, writer],
  tasks=[research_task, analysis_task, writing_task],
  process=Process.sequential
)

In [ ]:
# Execute the crew
result = report_crew.kickoff()

# Agent: Researcher
## Task: Generate relevant 50 rows about 2-3 random stocks in csv formate...


# Agent: Researcher
## Final Answer: 
Symbol,Name,Date,Opening Price,Closing Price,Volume Traded
AAPL,Apple Inc.,2023-01-02,150.00,148.76,9100000
GOOG,Google LLC (Alphabet),2023-01-02,2750.00,2742.50,3500000
TSLA,Tesla Inc.,2023-01-02,100.00,98.65,8000000
AMZN,Amazon Inc.,2023-01-02,105.00,107.25,4000000
FB,Facebook (Meta),2023-01-02,320.00,322.80,6000000
NFLX,Netflix Inc.,2023-01-02,500.00,495.75,3000000
MSFT,Microsoft Corporation,2023-01-02,250.00,255.60,7000000
COVIDX_Sector,-,--  // Sector specific index (example)
STOCKINDEX_US,,-, --, --,   // General U.S stock market performance indicator for reference on a single day from the sector standpoint without real-time data fetching tools:
VIX,Volatility Index of US Stocks,2023-01-02,18.50,19.75, -- // Not specific to individual stocks but relevant for understanding market sentiment
NASDAQ,Index,,,--  // Similar performance indicator witho

## Reference

1. https://docs.crewai.com/how-to/sequential-process
2. https://youtu.be/3tkmnItNXJM?si=jvAfWajDtBFWGO8C